In [1]:
pip install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

   
    app_name = runner_instance.app_name

   
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

   
    if user_queries:
       
        if type(user_queries) == str:
            user_queries = [user_queries]

       
        for query in user_queries:
            print(f"\nUser > {query}")

           
            query = types.Content(role="user", parts=[types.Part(text=query)])

           
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
               
                if event.content and event.content.parts:
                    
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  
)

# Session management
without proper context, they wont work properly

new prompt will forget prompt history
memory manegement is important
## session
container for memory management

## events and states
events are a building block of a convo

session.state is the Agent's scratchpad, where it stores and updates dynamic details needed during the conversation. Think of it as a global {key, value} pair storage which is available to all subagents and tools.


# session management
runner- Manages the flow of information between user and agent


Automatically maintains conversation history

In [6]:
APP_NAME = "default"  
USER_ID = "default"  
SESSION = "default"  

MODEL_NAME = "gemini-2.5-flash-lite"



root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot", 
)

session_service = InMemorySessionService()

#  Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [7]:
await run_session(
    runner,
    [
        "Hi, im pranav",
        "Hello! What is my name?",  
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, im pranav
gemini-2.5-flash-lite >  Hi Pranav! It's nice to meet you. How can I help you today?

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Pranav.


In [8]:
await run_session(
    runner,
    [
        "what did i ask before",  
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > what did i ask before
gemini-2.5-flash-lite >  You asked me "what did i ask before".


#  Persistent Sessions
let us use llm agent

In [9]:
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)


db_url = "sqlite:///my_agent_data.db" 
session_service = DatabaseSessionService(db_url=db_url)


runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


the overall outcomes to the query will be different

In [10]:
await run_session(
    runner, ["hi my name is pranav, what is my name?"], "test-db-session-01"
) 


 ### Session: test-db-session-01

User > hi my name is pranav, what is my name?
gemini-2.5-flash-lite >  Hi Pranav! Your name is Pranav.


In [11]:
await run_session(
    runner, ["what is my name?"], "test-db-session-02"
) 


 ### Session: test-db-session-02

User > what is my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, including your name. I am a text-based AI and do not store or recall any user data.


## check how the data is stored

In [12]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "hi my name is pranav, what is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Pranav! Your name is Pranav."}], "role": "model"}')
('default', 'test-db-session-02', 'user', '{"parts": [{"text": "what is my name?"}], "role": "user"}')
('default', 'test-db-session-02', 'text_chat_bot', '{"parts": [{"text": "I do not have access to your personal information, including your name. I am a text-based AI and do not store or recall any user data."}], "role": "model"}')


Context compaction, list of events turns morewhcih can lead to slower database
we create an object called app

In [13]:
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  
        overlap_size=1,
    ),
)

db_url = "sqlite:///my_agent_data.db" 
session_service = DatabaseSessionService(db_url=db_url)


research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

✅ Research App upgraded with Events Compaction!


/tmp/ipykernel_13/3187232874.py:5: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(
